# LAT NETs for the 2024 "Pivot to Chile"

July 2024

This notebook calculates NETs for the TMA-LAT, and the CD-LAT, both sited in Atacama.

The differences between these are:
- panel gaps in the CD-LAT (0.4% per mirror),
- 3 mirrors rather than 2 in the TMA-LAT, so a little more conductive and Ruze loss,
- different f/#'s for the cold aperture stop.

This modeling uses flat bands, and estimates the spillover on the aperture stop based on a Gaussian beam.  Psats are set to 3.0 times the calulated optical power in each band.

NETs are calculated per detector, per wafer, and "per tube", where horn-horn correlations, yield, the number of wafers/tube and the optically good fraction of detectors in a tube are all taken into account.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import yaml
from jbolo_funcs import *

%matplotlib inline
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = [12, 8]

In [5]:
!pwd

/Users/ruhl/code/bolo_calc_runs/jbolo/AoA/2024


In [2]:
!ls yamls/

Atacama-CDLAT_20240715.yaml           SAT_HWP_max_20240618.yaml
Atacama-TMALAT_20240715.yaml          SAT_baseline_20220726.yaml
SAT_HWP_AoAscaledoptics_20220726.yaml SAT_max_20240618.yaml
SAT_HWP_AoAscaledoptics_20240701.yaml


In [12]:
!diff yamls/Atacama-CDLAT_20240715.yaml yamls/Atacama-TMALAT_20240715.yaml

1c1
< version: {date: 20240715, name: 'Atacama-CDLAT'} 
---
> version: {date: 20240715, name: 'Atacama-TMALAT'}  
21,23c21,23
<   primary:      {obj_type: Mirror, conductivity: 36.9E6, surface_rough: 4.0E-6, spillover: 0.004}  # spill = gaps here.
<   secondary:    {obj_type: Mirror, conductivity: 36.9E6, surface_rough: 4.0E-6, spillover: 0.024}  # spill is 2% spill, 0.4% gaps
<   # 
---
>   primary:      {obj_type: Mirror, conductivity: 36.9E6, surface_rough: 4.0E-6}  # no spillover b/c no gaps
>   secondary:    {obj_type: Mirror, conductivity: 36.9E6, surface_rough: 4.0E-6}
>   tertiary:     {obj_type: Mirror, conductivity: 36.9E6, surface_rough: 4.0E-6, spillover: 0.02}
38,39c38
<   f_number: 2.0  # email from Pato to JR, MN and JcMc, 1/24/2022
<   #
---
>   f_number:  2.18 # (avg) Figure 7, "Freeform three-mirror anastigmatic large-aperture telescope and receiver optics for CMB-S4", P. Gallardo etal
158a158
> 


# Run CD-LAT

In [13]:
yamlfile = 'yamls/Atacama-CDLAT_20240715.yaml'
CDsim = yaml.safe_load(open(yamlfile))
# Run
run_optics(CDsim)
run_bolos(CDsim)
print_full_table(CDsim)


20240715 : Atacama-CDLAT
                              ULF       LF_1       LF_2       MF_1       MF_2       HF_1       HF_2
        det_bandcenter :    19.995     24.745     36.495     91.495    148.495    226.995    285.495  
         det_bandwidth :     4.990      6.490     16.990     28.990     40.990     57.990     58.990  
        sys_bandcenter :    20.174     24.994     37.483     92.748    149.609    227.667    285.671  
         sys_bandwidth :     4.092      5.256     12.634     22.969     35.466     54.749     58.244  
        sky_bandcenter :    20.166     25.001     37.420     92.795    149.552    227.521    285.215  
         sky_bandwidth :     4.140      5.240     13.049     23.023     36.251     55.139     57.723  
optics_effic_total_avg :     0.200      0.180      0.341      0.238      0.471      0.652      0.682  
        inst_effic_avg :     0.130      0.117      0.222      0.155      0.306      0.424      0.443  
         effic_tot_avg :     0.128      0.115      

# Run TMA-LAT

In [14]:
yamlfile = 'yamls/Atacama-TMALAT_20240715.yaml'
TMAsim = yaml.safe_load(open(yamlfile))
# Run
run_optics(TMAsim)
run_bolos(TMAsim)
print_full_table(TMAsim)


20240715 : Atacama-TMALAT
                              ULF       LF_1       LF_2       MF_1       MF_2       HF_1       HF_2
        det_bandcenter :    19.995     24.745     36.495     91.495    148.495    226.995    285.495  
         det_bandwidth :     4.990      6.490     16.990     28.990     40.990     57.990     58.990  
        sys_bandcenter :    20.178     24.999     37.526     92.783    149.704    227.836    285.787  
         sys_bandwidth :     4.073      5.233     12.449     22.807     34.997     53.865     57.631  
        sky_bandcenter :    20.170     25.006     37.464     92.830    149.646    227.689    285.330  
         sky_bandwidth :     4.121      5.217     12.857     22.861     35.774     54.243     57.912  
optics_effic_total_avg :     0.173      0.156      0.301      0.207      0.424      0.617      0.665  
        inst_effic_avg :     0.113      0.101      0.196      0.134      0.276      0.401      0.432  
         effic_tot_avg :     0.111      0.100     

In [25]:
print(' '.rjust(26),end='')
for ch in CDsim['channels'].keys():
    print(ch.rjust(11),end='')
print('')

print('CD: ',end='')
print_one_line(CDsim,'NET_C_wafer',1e6)
print('TMA:',end='')
print_one_line(TMAsim,'NET_C_wafer',1e6)
print()
print('CD: ',end='')
print_one_line(CDsim,'P_opt',1e12)
print('TMA:',end='')
print_one_line(TMAsim,'P_opt',1e12)

                                  ULF       LF_1       LF_2       MF_1       MF_2       HF_1       HF_2
CD:            NET_C_wafer :    82.494     59.262     31.374     13.566     12.885     27.178     65.778  
TMA:           NET_C_wafer :    82.230     59.783     31.036     13.467     12.624     27.101     65.044  

CD:                  P_opt :     0.158      0.190      1.020      1.375      4.160     11.967     17.422  
TMA:                 P_opt :     0.124      0.150      0.814      1.096      3.480     10.883     16.603  


In [22]:
print_optics(TMAsim,'HF_2')

HF_2
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6500    1.0000
Lens_3         :    0.0000     0.9839    0.6500
lyot           :    0.0000     0.9278    0.6395
lowpass        :    0.0000     0.9400    0.5933
Lens_2         :    0.0032     0.9839    0.5577
Lens_1         :    0.0031     0.9839    0.5487
nylon_filt     :    0.0896     0.9224    0.5398
alumina_filt   :    0.9545     0.9259    0.4979
RTMLI_10       :    0.4464     0.9917    0.4610
window         :    1.2817     0.9771    0.4572
tertiary       :    2.3394     0.9758    0.4467
secondary      :    0.3928     0.9958    0.4359
primary        :    0.3912     0.9958    0.4341
atmosphere     :   10.6721     0.8785    0.4323
cmb            :    0.0288     0.0000    0.3796
P_opticsonly_total = 5.9019e-12
P_optical_total =  1.6603e-11


In [26]:
print_optics(CDsim,'HF_2')

HF_2
Element            Popt(pW)   Effic  Effic_cumul
detector       :    0.0000     0.6500    1.0000
Lens_3         :    0.0000     0.9839    0.6500
lyot           :    0.0000     0.9555    0.6395
lowpass        :    0.0000     0.9400    0.6110
Lens_2         :    0.0032     0.9839    0.5744
Lens_1         :    0.0032     0.9839    0.5651
nylon_filt     :    0.0924     0.9224    0.5559
alumina_filt   :    0.9826     0.9259    0.5128
RTMLI_10       :    0.4597     0.9917    0.4748
window         :    1.3195     0.9771    0.4708
secondary      :    2.8080     0.9718    0.4600
primary        :    0.7904     0.9918    0.4471
atmosphere     :   10.9333     0.8785    0.4434
cmb            :    0.0296     0.0000    0.3895
P_opticsonly_total = 6.4591e-12
P_optical_total =  1.7422e-11
